# Computaional Theory Problems

# Problem 1 — Binary Words and Bitwise Operations

This notebook implements the core 32-bit logic functions defined in the **Secure Hash Standard (FIPS PUB 180-4, § 4.1.2)**.
These functions are used in the SHA-256 algorithm to combine 32-bit words with rotations, shifts, and Boolean operations.

The goal is to:
1. Build helper functions that safely handle 32-bit operations in NumPy.
2. Implement `Parity`, `Ch`, `Maj`, `Σ₀`, `Σ₁`, `σ₀`, and `σ₁`.
3. Document and explain each step clearly.


### Step 1 – Secure 32-bit Helpers
We use NumPy’s `uint32` type to enforce 32-bit behavior.
The helper functions `_to_u32`, `_rotr`, and `_shr` guarantee logical rotation and right-shift operations.


In [46]:
import numpy as np
from typing import Union

Word = np.uint32

def _to_u32(x: Union[int, np.integer]) -> Word:
    """Force value to 32-bit unsigned word."""
    return Word(int(x) & 0xFFFFFFFF)

def _rotr(x: Word, n: int) -> Word:
    """Rotate-right 32-bit x by n."""
    x = _to_u32(x); n = int(n) % 32
    if n == 0:
        return x
    return _to_u32((x >> n) | (x << Word(32 - n)))

def _shr(x: Word, n: int) -> Word:
    """Logical right shift 32-bit x by n (zero-fill)."""
    x = _to_u32(x); n = int(n) % 32
    return _to_u32(x >> n)

### Step 2 – Boolean Functions
These operations mix three 32-bit words using logical bit rules:
- **Parity(x,y,z)** = x ⊕ y ⊕ z  
- **Ch(x,y,z)** = (x ∧ y) ⊕ (¬x ∧ z)  
- **Maj(x,y,z)** = (x ∧ y) ⊕ (x ∧ z) ⊕ (y ∧ z)


In [47]:
def Parity(x, y, z) -> Word:
    """Parity(x,y,z): XOR from 32-bit ( SHA-1 parity)."""
    return _to_u32(_to_u32(x) ^ _to_u32(y) ^ _to_u32(z))

def Ch(x, y, z) -> Word:
    """Ch (Eq. 4.2): (x & y) ^ (~x & z) — pick bit from y if bit x =1, otherwise z."""
    x, y, z = _to_u32(x), _to_u32(y), _to_u32(z)
    return _to_u32((x & y) ^ ((~x) & z))

def Maj(x, y, z) -> Word:
    """Maj (Eq. 4.3): (x & y) ^ (x & z) ^ (y & z) — majority base on each bit."""
    x, y, z = _to_u32(x), _to_u32(y), _to_u32(z)
    return _to_u32((x & y) ^ (x & z) ^ (y & z))


### Step 3 – Σ and σ Functions
These are rotation/shift-based transformations defined in FIPS 180-4:
- **Σ₀(x)** = ROTR² ⊕ ROTR¹³ ⊕ ROTR²²  
- **Σ₁(x)** = ROTR⁶ ⊕ ROTR¹¹ ⊕ ROTR²⁵  
- **σ₀(x)** = ROTR⁷ ⊕ ROTR¹⁸ ⊕ SHR³  
- **σ₁(x)** = ROTR¹⁷ ⊕ ROTR¹⁹ ⊕ SHR¹⁰


In [48]:
def Sigma0(x) -> Word:
    """Σ₀: ROTR² ⊕ ROTR¹³ ⊕ ROTR²²."""
    x = _to_u32(x)
    return _to_u32(_rotr(x, 2) ^ _rotr(x, 13) ^ _rotr(x, 22))

def Sigma1(x) -> Word:
    """Σ₁: ROTR⁶ ⊕ ROTR¹¹ ⊕ ROTR²⁵."""
    x = _to_u32(x)
    return _to_u32(_rotr(x, 6) ^ _rotr(x, 11) ^ _rotr(x, 25))

def sigma0(x) -> Word:
    """σ₀: ROTR⁷ ⊕ ROTR¹⁸ ⊕ SHR³."""
    x = _to_u32(x)
    return _to_u32(_rotr(x, 7) ^ _rotr(x, 18) ^ _shr(x, 3))

def sigma1(x) -> Word:
    """σ₁: ROTR¹⁷ ⊕ ROTR¹⁹ ⊕ SHR¹⁰."""
    x = _to_u32(x)
    return _to_u32(_rotr(x, 17) ^ _rotr(x, 19) ^ _shr(x, 10))


### Step 4 – Demonstration with Fixed Values
We test the functions using example 32-bit words from the SHA-256 initial values.


In [49]:
# Predefined 32-bit demo inputs
x = np.uint32(0x6a09e667)  # constant from SHA-256 IV
y = np.uint32(0x12345678)
z = np.uint32(0xdeadbeef)

print("===== INPUT VALUES =====")
print(f"x = {hex(int(x))}")
print(f"y = {hex(int(y))}")
print(f"z = {hex(int(z))}")

print("\n===== LOGIC FUNCTIONS =====")
print(f"Parity(x, y, z) = {hex(int(Parity(x, y, z)))}")
print(f"Ch(x, y, z)     = {hex(int(Ch(x, y, z)))}")
print(f"Maj(x, y, z)    = {hex(int(Maj(x, y, z)))}")

print("\n===== SIGMA FUNCTIONS =====")
print(f"Sigma0(x) = {hex(int(Sigma0(x)))}")
print(f"Sigma1(x) = {hex(int(Sigma1(x)))}")
print(f"sigma0(x) = {hex(int(sigma0(x)))}")
print(f"sigma1(x) = {hex(int(sigma1(x)))}")


===== INPUT VALUES =====
x = 0x6a09e667
y = 0x12345678
z = 0xdeadbeef

===== LOGIC FUNCTIONS =====
Parity(x, y, z) = 0xa6900ef0
Ch(x, y, z)     = 0x96a45ee8
Maj(x, y, z)    = 0x5a2df66f

===== SIGMA FUNCTIONS =====
Sigma0(x) = 0xce20b47e
Sigma1(x) = 0x55b65510
sigma0(x) = 0xba0cf582
sigma1(x) = 0xcfe5da3c


### Step 5 – Reflection and Research Discussion
According to **FIPS PUB 180-4** (NIST, 2015), these functions form the
non-linear mixing stage of SHA-256.  
Each rotation and shift ensures diffusion and bit independence.

Sources:
- [NIST FIPS 180-4 (2015) — Secure Hash Standard](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)
- Numpy documentation on [Unsigned integer types](https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.uint32)


## Problem 2: Fractional Parts of Cube Roots
This script computes the 64 SHA-256 constants K[0..63] defined in the
Secure Hash Standard (FIPS PUB 180-4, §4.2.2).

Each constant is the first 32 bits of the fractional part of the cube root
of the first 64 prime numbers.

### Step 1: Generate the first 64 prime numbers

In [50]:
import numpy as np
from typing import List

def primes(n: int) -> np.ndarray:
    """
    Generate the first n prime numbers.
    Parameters
    n : int
        The number of prime numbers to generate.
    np.ndarray
        Array of the first n primes as Python integers.
    The upper bound is expanded if needed.
    """
    if n < 1:
        return np.array([], dtype=int)

    # Rough upper bound for nth prime
    if n < 6:
        bound = 15
    else:
        nf = float(n)
        bound = int(nf * (np.log(nf) + np.log(np.log(nf))) + 50)

    def sieve(limit: int) -> List[int]:
        arr = np.ones(limit + 1, dtype=bool)
        arr[:2] = False
        for p in range(2, int(limit**0.5) + 1):
            if arr[p]:
                arr[p*p:limit+1:p] = False
        return np.flatnonzero(arr).tolist()

    ps = sieve(bound)
    while len(ps) < n:
        bound *= 2
        ps = sieve(bound)

    return np.array(ps[:n], dtype=int)

### Step 2: Cube roots of the first 64 primes

    1. Generate n primes.

    2. Compute cube roots using NumPy (float64 precision).

    3. Take fractional part (root − floor(root)).

    4. Multiply by 2^32 and floor to integer.

    5. Cast to np.uint32 to enforce 32-bit size.
    

In [51]:
def cube_root_constants(n: int = 64) -> np.ndarray:
    """
    Compute 32-bit constants from fractional parts of cube roots of primes.
    """
    p = primes(n).astype(np.float64)
    roots = np.cbrt(p)                           # cube roots
    frac  = roots - np.floor(roots)              # fractional part
    scaled = np.floor(frac * (2**32))            # bring first 32 bits in front
    return scaled.astype(np.uint32)

### Step 3: Display results in hex and verify

In [52]:
def display_constants(k_values: np.ndarray) -> None:
    """
    Print constants in 8-digit hexadecimal form and verify against
    """
    hex_vals = [f"{int(v):08x}" for v in k_values]

    # Official reference values from FIPS 180-4 § 4.2.2
    ref_hex = [
        "428a2f98","71374491","b5c0fbcf","e9b5dba5","3956c25b","59f111f1","923f82a4","ab1c5ed5",
        "d807aa98","12835b01","243185be","550c7dc3","72be5d74","80deb1fe","9bdc06a7","c19bf174",
        "e49b69c1","efbe4786","0fc19dc6","240ca1cc","2de92c6f","4a7484aa","5cb0a9dc","76f988da",
        "983e5152","a831c66d","b00327c8","bf597fc7","c6e00bf3","d5a79147","06ca6351","14292967",
        "27b70a85","2e1b2138","4d2c6dfc","53380d13","650a7354","766a0abb","81c2c92e","92722c85",
        "a2bfe8a1","a81a664b","c24b8b70","c76c51a3","d192e819","d6990624","f40e3585","106aa070",
        "19a4c116","1e376c08","2748774c","34b0bcb5","391c0cb3","4ed8aa4a","5b9cca4f","682e6ff3",
        "748f82ee","78a5636f","84c87814","8cc70208","90befffa","a4506ceb","bef9a3f7","c67178f2",
    ]

    matches = [calc == ref for calc, ref in zip(hex_vals, ref_hex)]
    all_match = all(matches)

    print("===== SHA-256 Constants (K[0..63]) =====")
    for i, hx in enumerate(hex_vals):
        print(f"K[{i:02}] = 0x{hx}")
    print("\nAll 64 constants match FIPS 180-4 reference:", all_match)

### Step 4: Main execution

In [53]:
if __name__ == "__main__":
    K = cube_root_constants(64)
    display_constants(K)

===== SHA-256 Constants (K[0..63]) =====
K[00] = 0x428a2f98
K[01] = 0x71374491
K[02] = 0xb5c0fbcf
K[03] = 0xe9b5dba5
K[04] = 0x3956c25b
K[05] = 0x59f111f1
K[06] = 0x923f82a4
K[07] = 0xab1c5ed5
K[08] = 0xd807aa98
K[09] = 0x12835b01
K[10] = 0x243185be
K[11] = 0x550c7dc3
K[12] = 0x72be5d74
K[13] = 0x80deb1fe
K[14] = 0x9bdc06a7
K[15] = 0xc19bf174
K[16] = 0xe49b69c1
K[17] = 0xefbe4786
K[18] = 0x0fc19dc6
K[19] = 0x240ca1cc
K[20] = 0x2de92c6f
K[21] = 0x4a7484aa
K[22] = 0x5cb0a9dc
K[23] = 0x76f988da
K[24] = 0x983e5152
K[25] = 0xa831c66d
K[26] = 0xb00327c8
K[27] = 0xbf597fc7
K[28] = 0xc6e00bf3
K[29] = 0xd5a79147
K[30] = 0x06ca6351
K[31] = 0x14292967
K[32] = 0x27b70a85
K[33] = 0x2e1b2138
K[34] = 0x4d2c6dfc
K[35] = 0x53380d13
K[36] = 0x650a7354
K[37] = 0x766a0abb
K[38] = 0x81c2c92e
K[39] = 0x92722c85
K[40] = 0xa2bfe8a1
K[41] = 0xa81a664b
K[42] = 0xc24b8b70
K[43] = 0xc76c51a3
K[44] = 0xd192e819
K[45] = 0xd6990624
K[46] = 0xf40e3585
K[47] = 0x106aa070
K[48] = 0x19a4c116
K[49] = 0x1e376c08
K[50] = 0

## Problem 3: Padding

In [38]:
1

1

## Problem 4: Hashes

In [39]:
1

1

## Problem 5: Passwords

In [40]:
1

1